# 具体的な投資方法
    S＆P500
        S&P500は、短期SMA39と長期SMA267が最もパフォーマンスがよかったので採用とする。
        また、米国株は米国債券と強い逆相関が認められるため、米国債券で買いのフラグが立っているときは様子見とする。
        米国債券の売買判定は、短期SMA20と長期SMA50を利用。

        条件：（S&P500で短期SMA39と長期SMA267で買いフラグ）　＆　（米国債券で短期SMA20と長期SMA50で売りフラグ）

        取引のタイミングは、米国市場のCLOSE時間。
    
    GOLD
        GOLDは、どのSMA戦略も一長一短であったため、3つのSMA戦略が一致したときのみ買いとする。
        思想としては、短期目線と中期目線と長期目線で方向が一致していれば安定するだろうというもの。

        条件：（短期SMA3と長期SMA5で買いフラグ）　＆　（短期SMA86と長期SMA133で買いフラグ）　＆　（短期SMA91と長期SMA201で買いフラグ）

In [8]:
import pandas as pd
import numpy as np
import yfinance as yf

def makeSma(meigara, tankiSma, choukiSma):

    # S&P500のデータをyfinanceから取得
    spx = yf.download(meigara, start='2000-01-01', end='2050-03-04')

    # 短期移動平均線を計算
    spx['短期SMA'] = spx['Adj Close'].rolling(window=tankiSma).mean()

    # 長期移動平均線を計算
    spx['長期SMA'] = spx['Adj Close'].rolling(window=choukiSma).mean()

    # ポジションを保持するための変数
    position = 0

    #ポジションをとった時のPrice
    numPositon = 0    

    # 取引履歴を保存するDataFrame
    trades = pd.DataFrame(columns=['Date', 'Position', 'Price'])

    # 移動平均線を用いたバックテストを実施
    for i in range(len(spx)):
        # 短期移動平均線が長期移動平均線を上抜けした場合、買いポジションを取る
        if spx['短期SMA'][i] > spx['長期SMA'][i] and position == 0:
            position = 1
            trades = trades.append({'Date': spx.index[i], 'Position': 'Buy', 'Price': spx['Adj Close'][i]}, ignore_index=True)

        # 短期移動平均線が長期移動平均線を下抜けした場合、決済
        elif spx['短期SMA'][i] < spx['長期SMA'][i] and position == 1:
            position = 0
            trades = trades.append({'Date': spx.index[i], 'Position': 'Sell', 'Price': spx['Adj Close'][i]}, ignore_index=True)
    
    #決済金額を追記
    trades['result']=0

    for i in range(len(trades)):
        #Buyなら買う
        if trades.iloc[i, 1] == 'Buy':
            numPositon = trades.iloc[i, 2]
        #Sellなら決済
        if trades.iloc[i, 1] == 'Sell':
            trades.iloc[i, 3] = (trades.iloc[i, 2] - numPositon) / numPositon



    # # 取引履歴を表示
    # print(trades)

    # csvFileName = 'ChatGPT_' + meigara + '_SMA.csv'

    # #CSV出力
    # trades.to_csv(csvFileName, index=False)

    return trades


def backTest(df):
    df['result'] = 0
    positionflg = 0
    positionNum = 0

    #ポジションが無くて、flgがBuyなら買う。
    for i in range(len(df)):
        if positionflg == 0:
            if df['flg'].iloc[i] == 'Buy':
                positionNum = df['Close'].iloc[i]
                positionflg = 1

        #ポジションがあって、flgがないなら決済
        else:
            if df['flg'].iloc[i] == '':
                df['result'].iloc[i] = (df['Close'].iloc[i] - positionNum) / df['Close'].iloc[i]
                positionflg = 0
    
    return df




# ----------------------------------------------------------------------------------------------------------------------------
# S&P500の判定
# ----------------------------------------------------------------------------------------------------------------------------

tradeslist = [39,267]
trades_select_GSPC = makeSma(meigara='^GSPC', tankiSma=tradeslist[0], choukiSma=tradeslist[1])

tradeslist = [20,50]
trades_select_TNX = makeSma(meigara='^TNX', tankiSma=tradeslist[0], choukiSma=tradeslist[1])

#最新の投資判断出力
print(trades_select_GSPC[-1:])
print(trades_select_TNX[-1:])

#S＆P500を読み込む
df_gspc = yf.download('^GSPC', start='2000-01-01', end='2050-03-04')

#新規に列を作成
df_gspc['flgSMA'] = ''
df_gspc['flgTNX'] = ''
df_gspc['flg'] = ''

#Dateをindexから列に変換
df_gspc.reset_index(inplace=True)

#Dateを日付に変換
df_gspc['Date'] = pd.to_datetime(df_gspc['Date'])
trades_select_GSPC['Date'] = pd.to_datetime(trades_select_GSPC['Date'])
trades_select_TNX['Date'] = pd.to_datetime(trades_select_TNX['Date'])

#SMAのフラグを付ける
for i in range(len(trades_select_GSPC)):
    if trades_select_GSPC['Position'].iloc[i] == "Buy":
        df_gspc.loc[(df_gspc['Date'] >= trades_select_GSPC['Date'].iloc[i]), 'flgSMA'] = 'Buy'

    if trades_select_GSPC['Position'].iloc[i] == "Sell":
        df_gspc.loc[(df_gspc['Date'] >= trades_select_GSPC['Date'].iloc[i]), 'flgSMA'] = ''

#米国債のフラグを付ける
for i in range(len(trades_select_TNX)):
    if trades_select_TNX['Position'].iloc[i] == "Sell":
        df_gspc.loc[(df_gspc['Date'] >= trades_select_TNX['Date'].iloc[i]), 'flgTNX'] = 'Sell'

    if trades_select_TNX['Position'].iloc[i] == "Buy":
        df_gspc.loc[(df_gspc['Date'] >= trades_select_TNX['Date'].iloc[i]), 'flgTNX'] = ''

#最終的な売買フラグを立てる
df_gspc.loc[(df_gspc['flgSMA'] == 'Buy') & (df_gspc['flgTNX'] == 'Sell'), 'flg'] = 'Buy'

#バックテスト実施
df_gspc = backTest(df_gspc)

#バックテスト結果の合計
print(df_gspc['result'].sum())

#バックテスト結果出力
with pd.ExcelWriter('./output/SMA投資法.xlsx',mode='a', if_sheet_exists='overlay') as writer:
    df_gspc.to_excel(writer, sheet_name='SP500', index=False)


# ----------------------------------------------------------------------------------------------------------------------------
# GOLDの判定
# ----------------------------------------------------------------------------------------------------------------------------
tradeslist = [3,5]
trades_select_GOLD1 = makeSma(meigara='GOLD', tankiSma=tradeslist[0], choukiSma=tradeslist[1])

tradeslist = [86,133]
trades_select_GOLD2 = makeSma(meigara='GOLD', tankiSma=tradeslist[0], choukiSma=tradeslist[1])

tradeslist = [91,201]
trades_select_GOLD3 = makeSma(meigara='GOLD', tankiSma=tradeslist[0], choukiSma=tradeslist[1])

#投資判断の出力
print(trades_select_GOLD1[-1:])
print(trades_select_GOLD2[-1:])
print(trades_select_GOLD3[-1:])

#GOLDを読み込む
df_gold = yf.download('GOLD', start='2000-01-01', end='2050-03-04')

#新規に列を作成
df_gold['flgSMA1'] = ''
df_gold['flgSMA2'] = ''
df_gold['flgSMA3'] = ''
df_gold['flg'] = ''

#Dateをindexから列に変換
df_gold.reset_index(inplace=True)

#Dateを日付に変換
df_gold['Date'] = pd.to_datetime(df_gold['Date'])
trades_select_GOLD1['Date'] = pd.to_datetime(trades_select_GOLD1['Date'])
trades_select_GOLD2['Date'] = pd.to_datetime(trades_select_GOLD2['Date'])
trades_select_GOLD3['Date'] = pd.to_datetime(trades_select_GOLD3['Date'])

#SMA1のフラグを付ける
for i in range(len(trades_select_GOLD1)):
    if trades_select_GOLD1['Position'].iloc[i] == "Buy":
        df_gold.loc[(df_gold['Date'] >= trades_select_GOLD1['Date'].iloc[i]), 'flgSMA1'] = 'Buy'

    if trades_select_GOLD1['Position'].iloc[i] == "Sell":
        df_gold.loc[(df_gold['Date'] >= trades_select_GOLD1['Date'].iloc[i]), 'flgSMA1'] = ''

#SMA2のフラグを付ける
for i in range(len(trades_select_GOLD2)):
    if trades_select_GOLD2['Position'].iloc[i] == "Buy":
        df_gold.loc[(df_gold['Date'] >= trades_select_GOLD2['Date'].iloc[i]), 'flgSMA2'] = 'Buy'

    if trades_select_GOLD2['Position'].iloc[i] == "Sell":
        df_gold.loc[(df_gold['Date'] >= trades_select_GOLD2['Date'].iloc[i]), 'flgSMA2'] = ''

#SMA3のフラグを付ける
for i in range(len(trades_select_GOLD3)):
    if trades_select_GOLD3['Position'].iloc[i] == "Buy":
        df_gold.loc[(df_gold['Date'] >= trades_select_GOLD3['Date'].iloc[i]), 'flgSMA3'] = 'Buy'

    if trades_select_GOLD3['Position'].iloc[i] == "Sell":
        df_gold.loc[(df_gold['Date'] >= trades_select_GOLD3['Date'].iloc[i]), 'flgSMA3'] = ''

#最終的な売買フラグを立てる
df_gold.loc[(df_gold['flgSMA1'] == 'Buy') & (df_gold['flgSMA2'] == 'Buy') & (df_gold['flgSMA3'] == 'Buy'), 'flg'] = 'Buy'

#バックテスト実施
df_gold = backTest(df_gold)

#バックテスト結果の合計
print(df_gold['result'].sum())

#バックテスト結果出力
with pd.ExcelWriter('./output/SMA投資法.xlsx',mode='a', if_sheet_exists='overlay') as writer:
    df_gold.to_excel(writer, sheet_name='GOLD', index=False)

[*********************100%***********************]  1 of 1 completed


C:\Users\swdtk\AppData\Local\Temp\ipykernel_4268\29748267.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append({'Date': spx.index[i], 'Position': 'Buy', 'Price': spx['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_4268\29748267.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append({'Date': spx.index[i], 'Position': 'Sell', 'Price': spx['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_4268\29748267.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append({'Date': spx.index[i], 'Position': 'Buy', 'Price': spx['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_4268\29748

[*********************100%***********************]  1 of 1 completed
         Date Position        Price  result
18 2023-03-07      Buy  3986.370117     0.0
          Date Position  Price  result
120 2023-02-17      Buy  3.828     0.0


C:\Users\swdtk\AppData\Local\Temp\ipykernel_4268\29748267.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append({'Date': spx.index[i], 'Position': 'Buy', 'Price': spx['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_4268\29748267.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append({'Date': spx.index[i], 'Position': 'Sell', 'Price': spx['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_4268\29748267.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append({'Date': spx.index[i], 'Position': 'Buy', 'Price': spx['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_4268\29748

[*********************100%***********************]  1 of 1 completed
1.1166210211038698


...

[*********************100%***********************]  1 of 1 completed
           Date Position  Price    result
1277 2023-03-07     Sell  15.77 -0.033108
         Date Position   Price  result
60 2023-01-06      Buy  18.921     0.0
         Date Position  Price  result
24 2023-02-24      Buy   15.9     0.0


C:\Users\swdtk\AppData\Local\Temp\ipykernel_4268\29748267.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append({'Date': spx.index[i], 'Position': 'Buy', 'Price': spx['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_4268\29748267.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append({'Date': spx.index[i], 'Position': 'Sell', 'Price': spx['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_4268\29748267.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trades = trades.append({'Date': spx.index[i], 'Position': 'Buy', 'Price': spx['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_4268\29748

[*********************100%***********************]  1 of 1 completed


C:\Users\swdtk\AppData\Local\Temp\ipykernel_4268\29748267.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['result'].iloc[i] = (df['Close'].iloc[i] - positionNum) / df['Close'].iloc[i]


1.037527918310513


In [7]:
#バックテスト結果出力
with pd.ExcelWriter('./output/SMA投資法.xlsx',mode='a', if_sheet_exists='overlay') as writer:
    df_gold.to_excel(writer, sheet_name='GOLD', index=False)

In [5]:

#バックテスト結果出力
df_gold.to_excel('./output/SMA投資法.xlsx', sheet_name='GOLD', index=False)

In [3]:
yf.download('^TNX', start='2000-01-01', end='2050-03-04')

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,6.498,6.603,6.498,6.548,6.548,0
2000-01-04,6.530,6.548,6.485,6.485,6.485,0
2000-01-05,6.521,6.599,6.508,6.599,6.599,0
2000-01-06,6.558,6.585,6.540,6.549,6.549,0
2000-01-07,6.545,6.595,6.504,6.504,6.504,0
...,...,...,...,...,...,...
2023-03-01,3.940,4.006,3.934,3.994,3.994,0
2023-03-02,4.050,4.091,4.044,4.073,4.073,0
2023-03-03,4.013,4.026,3.962,3.964,3.964,0


In [4]:
yf.download('GOLD', start='2000-01-01', end='2050-03-04')

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,17.562500,18.1875,17.437500,17.687500,12.784675,1509900
2000-01-04,17.312500,17.6875,17.125000,17.437500,12.603970,1787100
2000-01-05,17.562500,17.8750,17.437500,17.750000,12.829847,1285100
2000-01-06,17.750000,17.8125,17.500000,17.750000,12.829847,1139300
2000-01-07,17.812500,17.8750,17.250000,17.312500,12.513617,1199900
...,...,...,...,...,...,...
2023-03-01,16.430000,16.4800,16.180000,16.309999,16.309999,14867000
2023-03-02,16.190001,16.4000,16.120001,16.379999,16.379999,13362100
2023-03-03,16.559999,16.5700,16.340000,16.520000,16.520000,14362100


In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [8]:
# 日経225のデータをyfinanceから取得（2000/1/1～2018/12/31までを取得）
n225_train = yf.download('^N225', start='2000-01-01', end='2018-12-31')

# 参考（ソフトバンクの株価を取りたいときは下記）
# sb_train = yf.download('9434.T', start='2019-01-01', end='2050-03-04')

[*********************100%***********************]  1 of 1 completed


In [9]:
n225_train

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-04,18937.449219,19187.609375,18937.449219,19002.859375,19002.859375,0
2000-01-05,19003.509766,19003.509766,18221.820312,18542.550781,18542.550781,0
2000-01-06,18574.009766,18582.740234,18168.269531,18168.269531,18168.269531,0
2000-01-07,18194.050781,18285.730469,18068.099609,18193.410156,18193.410156,0
2000-01-11,18246.099609,18887.560547,18246.099609,18850.919922,18850.919922,0
...,...,...,...,...,...,...
2018-12-21,20310.500000,20334.730469,20006.669922,20166.189453,20166.189453,116600000
2018-12-25,19785.429688,19785.429688,19117.960938,19155.740234,19155.740234,0
2018-12-26,19302.589844,19530.349609,18948.580078,19327.060547,19327.060547,79900000


In [16]:
def makeSma(dataDf, tankiSma, choukiSma):
    """
    関数の説明：
        yfinanceで取得してきた株価のDataFrameを基に、
        ゴールデンクロスなら買い、デッドクロスなら決済のフラグを作成してくれる関数。
    
    Args:
        dataDf: yfinanceで取得してきた株価のDataFrame
        tankiSma: 短期移動平均線の日数
        choukiSma: 長期移動平均線の日数

    Returns:
        tradesDf: 売買フラグを付与したDataFrameを返す。
    """
    # 短期移動平均線を計算
    dataDf['短期SMA'] = dataDf['Adj Close'].rolling(window=tankiSma).mean()

    # 長期移動平均線を計算
    dataDf['長期SMA'] = dataDf['Adj Close'].rolling(window=choukiSma).mean()

    # ポジションを保持するための変数
    position = 0

    #ポジションをとった時のPrice
    numPositon = 0    

    # 取引履歴を保存するDataFrame
    tradesDf = pd.DataFrame(columns=['Date', 'Position', 'Price'])

    # 移動平均線を用いたバックテストを実施
    for i in range(len(dataDf)):
        # 短期移動平均線が長期移動平均線を上抜けした場合、買いポジションを取る
        if dataDf['短期SMA'][i] > dataDf['長期SMA'][i] and position == 0:
            position = 1
            tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)

        # 短期移動平均線が長期移動平均線を下抜けした場合、決済
        elif dataDf['短期SMA'][i] < dataDf['長期SMA'][i] and position == 1:
            position = 0
            tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
    
    #決済金額を追記
    tradesDf['result']=0

    for i in range(len(tradesDf)):
        #Buyなら買う
        if tradesDf.iloc[i, 1] == 'Buy':
            numPositon = tradesDf.iloc[i, 2]
        #Sellなら決済
        if tradesDf.iloc[i, 1] == 'Sell':
            tradesDf.iloc[i, 3] = (tradesDf.iloc[i, 2] - numPositon) / numPositon

    # 取引履歴を表示
    print(tradesDf)

    csvFileName = 'SystemTrade' + '_SMA.csv'

    #CSV出力
    tradesDf.to_csv(csvFileName, index=False)

    return tradesDf

def backTest(df):
    """
    関数の説明：
        売買取引のバックテスト用関数

    Args:
        df: 売買取引させる

    Returns:
        df: 決済後のDataFrame
    """
    df['result'] = 0
    positionflg = 0
    positionNum = 0

    #ポジションが無くて、flgがBuyなら買う。
    for i in range(len(df)):
        if positionflg == 0:
            if df['flg'].iloc[i] == 'Buy':
                positionNum = df['Close'].iloc[i]
                positionflg = 1

        #ポジションがあって、flgがないなら決済
        else:
            if df['flg'].iloc[i] == '':
                df['result'].iloc[i] = (df['Close'].iloc[i] - positionNum) / df['Close'].iloc[i]
                positionflg = 0
    
    return df

In [29]:
n225_sma = makeSma(dataDf=n225_train, tankiSma=5, choukiSma=25)

C:\Users\swdtk\AppData\Local\Temp\ipykernel_9128\714882119.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_9128\714882119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_9128\714882119.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppDat

          Date Position         Price    result
0   2000-02-08      Buy  19868.880859  0.000000
1   2000-03-13     Sell  19189.929688 -0.034172
2   2000-03-24      Buy  19958.080078  0.000000
3   2000-04-18     Sell  18969.519531 -0.049532
4   2000-06-08      Buy  17004.339844  0.000000
..         ...      ...           ...       ...
235 2018-10-12     Sell  22694.660156 -0.005550
236 2018-11-12      Buy  22269.880859  0.000000
237 2018-11-15     Sell  21803.619141 -0.020937
238 2018-11-29      Buy  22262.599609  0.000000
239 2018-12-07     Sell  21678.679688 -0.026229

[240 rows x 4 columns]


C:\Users\swdtk\AppData\Local\Temp\ipykernel_9128\714882119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_9128\714882119.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Buy', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppData\Local\Temp\ipykernel_9128\714882119.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tradesDf = tradesDf.append({'Date': dataDf.index[i], 'Position': 'Sell', 'Price': dataDf['Adj Close'][i]}, ignore_index=True)
C:\Users\swdtk\AppDa

In [30]:
#新規に列を作成
n225_train['flg'] = ''

#Dateをindexから列に変換
n225_train.reset_index(inplace=True)

#Dateを日付に変換
n225_train['Date'] = pd.to_datetime(n225_train['Date'])
n225_sma['Date'] = pd.to_datetime(n225_sma['Date'])

#SMAのフラグを付ける
for i in range(len(n225_sma)):
    if n225_sma['Position'].iloc[i] == "Buy":
        n225_train.loc[(n225_train['Date'] >= n225_sma['Date'].iloc[i]), 'flg'] = 'Buy'

    if n225_sma['Position'].iloc[i] == "Sell":
        n225_train.loc[(n225_train['Date'] >= n225_sma['Date'].iloc[i]), 'flg'] = ''

In [33]:
n225_train['flg'].unique()

array(['', 'Buy'], dtype=object)

In [34]:
n225_train

,Date,Open,High,Low,Close,Adj Close,Volume,短期SMA,長期SMA,flg
0,2000-01-04,18937.449219,19187.609375,18937.449219,19002.859375,19002.859375,0,NaN,NaN,
1,2000-01-05,19003.509766,19003.509766,18221.820312,18542.550781,18542.550781,0,NaN,NaN,
2,2000-01-06,18574.009766,18582.740234,18168.269531,18168.269531,18168.269531,0,NaN,NaN,
3,2000-01-07,18194.050781,18285.730469,18068.099609,18193.410156,18193.410156,0,NaN,NaN,
4,2000-01-11,18246.099609,18887.560547,18246.099609,18850.919922,18850.919922,0,18551.601953,NaN,
...,...,...,...,...,...,...,...,...,...,...
4658,2018-12-21,20310.500000,20334.730469,20006.669922,20166.189453,20166.189453,116600000,20833.803906,21593.381484,
4659,2018-12-25,19785.429688,19785.429688,19117.960938,19155.740234,19155.740234,0,20363.575781,21492.397500,
4660,2018-12-26,19302.589844,19530.349609,18948.580078,19327.060547,19327.060547,79900000,20005.898047,21392.633516,
4661,2018-12-27,19706.189453,20211.570312,19701.759766,20077.619141,20077.619141,93000000,19823.837891,21332.413516,


In [36]:
n225_sma['result'].sum()

0.5362064095993865

In [37]:
columns = ['短期SMA', '長期SMA', '結果']
resultDf = pd.DataFrame(columns=columns)

In [38]:
resultDf

,短期SMA,長期SMA,結果


In [40]:
# 新しい行を追加
new_row = {'短期SMA': 1, '長期SMA': 2, '結果': n225_sma['result'].sum()}
resultDf = resultDf.append(new_row, ignore_index=True)

C:\Users\swdtk\AppData\Local\Temp\ipykernel_9128\3298725188.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultDf = resultDf.append(new_row, ignore_index=True)


In [41]:
resultDf

,短期SMA,長期SMA,結果
0,1.0,2.0,0.536206
